# Test WaffleIron Model

In [1]:
from models.waffleiron.segmenter import Segmenter

Using torch.scatter_reduce for 3D to 2D projection.
Using torch.scatter_reduce for 3D to 2D projection.


In [2]:
model = Segmenter(
    input_channels=5,
    feat_channels=768,
    depth=48,
    grid_shape=[[256, 256], [256, 32], [256, 32]],
    nb_class=16,
    #drop_path_prob=config["waffleiron"]["drop_path"],
    layer_norm=True,
)

In [3]:
import torch

# Load pretrained model
ckpt = torch.load('./saved_models/ckpt_last_scalr.pth', map_location="cpu")
ckpt = ckpt["net"]

In [4]:
print(ckpt.keys())

dict_keys(['module.embed.norm.weight', 'module.embed.norm.bias', 'module.embed.norm.running_mean', 'module.embed.norm.running_var', 'module.embed.norm.num_batches_tracked', 'module.embed.conv1.weight', 'module.embed.conv1.bias', 'module.embed.conv2.0.weight', 'module.embed.conv2.0.bias', 'module.embed.conv2.0.running_mean', 'module.embed.conv2.0.running_var', 'module.embed.conv2.0.num_batches_tracked', 'module.embed.conv2.1.weight', 'module.embed.conv2.2.weight', 'module.embed.conv2.2.bias', 'module.embed.conv2.2.running_mean', 'module.embed.conv2.2.running_var', 'module.embed.conv2.2.num_batches_tracked', 'module.embed.conv2.4.weight', 'module.embed.final.weight', 'module.embed.final.bias', 'module.waffleiron.channel_mix.0.scale.weight', 'module.waffleiron.channel_mix.0.norm.weight', 'module.waffleiron.channel_mix.0.norm.bias', 'module.waffleiron.channel_mix.0.mlp.0.weight', 'module.waffleiron.channel_mix.0.mlp.0.bias', 'module.waffleiron.channel_mix.0.mlp.2.weight', 'module.waffleiro

In [5]:
new_ckpt = {}
for k in ckpt.keys():
    if k.startswith("module"):
        if k.startswith("module.classif.0"):
            continue
        elif k.startswith("module.classif.1"):
            new_ckpt["classif" + k[len("module.classif.1") :]] = ckpt[k]
        else:
            new_ckpt[k[len("module.") :]] = ckpt[k]
    else:
        new_ckpt[k] = ckpt[k]

In [6]:
new_ckpt.get("classif.weight").shape

torch.Size([16, 768, 1])

In [7]:
model.load_state_dict(new_ckpt)

<All keys matched successfully>

## Model loaded --> Test features somehow

In [13]:
import os
import torch
from auxiliary.process_data.nuscenes.nuscenes_dataset import DatasetTrainVal

print("Creating dataloader...", flush=True)

target = '/root/main/dataset/processed'

filelist_train = [os.path.join(target, 'train_pointclouds', fname) for fname in os.listdir(os.path.join(target, 'train_pointclouds')) if os.path.splitext(fname)[1]==".npy"]
filelist_train.sort()
filelist_val = filelist_train[:3]
filelist_train = filelist_train[3:]

ds = DatasetTrainVal(filelist_train, os.path.join(target, 'train_pointclouds'),
                            training=True,
                            npoints=2000,
                            iteration_number=(2*10)*10,
                            jitter=0.2)
train_loader = torch.utils.data.DataLoader(ds, batch_size=1, shuffle=False, # Change batch_size
                                    num_workers=1)

Creating dataloader...
Len: 82


In [18]:
for t in train_loader:
    print(torch.bincount(t['target'][0]))
    x = input()

tensor([1244,  274,    0,    0,    0,    0,    0,  482])

tensor([1191,  376,    0,    0,    0,    0,    0,  433])

tensor([1149,  277,    0,    0,    0,    0,    0,  574])

tensor([1139,  272,    0,    0,    0,    0,    0,  589])

tensor([1059,  343,    0,    0,    0,    0,    0,  598])

tensor([1264,  448,    0,    0,    0,    0,    0,  288])

tensor([1208,   68,    0,    0,    0,    0,    0,  724])

tensor([1421,  133,    0,    0,    0,    0,    0,  446])

tensor([1095,  317,    0,    0,    0,    0,    0,  588])

tensor([1157,  145,    0,    0,    0,    0,    0,  698])

tensor([1089,  320,    0,    0,    0,    0,    0,  591])

tensor([1084,  284,    0,    0,    0,    0,    0,  632])

tensor([1059,  360,    0,    0,    0,    0,    0,  581])

tensor([1114,  247,    0,    0,    0,    0,    0,  639])

tensor([1138,  285,    0,    0,    0,    0,    0,  577])

tensor([994, 552,   0,   0,   0,   0,   0, 454])


KeyboardInterrupt: Interrupted by user

In [9]:
from plyfile import PlyData, PlyElement
import numpy as np
import os

train_filenames = ["Lille1_1.ply",  "Lille1_2.ply",  "Lille2.ply",  "Paris.ply",]
destdir = '/root/main/dataset/processed/'
rootdir = '/root/main/dataset/'

filenames = train_filenames
save_dir = os.path.join(destdir,"train_pointclouds")
pts_all = {}
for filename in filenames:
    fname = os.path.join(rootdir, "training_10_classes", filename)
    plydata = PlyData.read(fname)
    x = plydata["vertex"].data["x"].astype(np.float32)
    y = plydata["vertex"].data["y"].astype(np.float32)
    z = plydata["vertex"].data["z"].astype(np.float32)
    reflectance = plydata["vertex"].data["reflectance"].astype(np.float32)
    label = plydata["vertex"].data["class"].astype(np.float32)
    pts = np.concatenate([
        np.expand_dims(x,1),
        np.expand_dims(y,1),
        np.expand_dims(z,1),
        np.expand_dims(reflectance,1),
        np.expand_dims(label,1),
    ], axis=1).astype(np.float32)
    pts_all[filename] = pts

In [10]:
pts_all["Lille1_1.ply"][:,1]

array([  83.03686 ,   83.05253 ,   83.117584, ..., -386.54715 ,
       -386.5629  , -386.97958 ], dtype=float32)

In [11]:
print(min(pts_all["Lille1_1.ply"][:,0]))
print(max(pts_all["Lille1_1.ply"][:,0]))
print(min(pts_all["Lille1_1.ply"][:,1]))
print(max(pts_all["Lille1_1.ply"][:,1]))
print(min(pts_all["Lille1_1.ply"][:,2]))
print(max(pts_all["Lille1_1.ply"][:,2]))

-360.1785
67.185555
-412.13937
101.893524
32.78496
66.74454


In [13]:
(101.893524 - (-412.13937))/20 

25.701644699999996

In [14]:
20 * 25 * 3

1500

In [16]:
def voxelize_pointcloud(points, voxel_size):
    """
    Voxelizes a point cloud.

    Args:
        points (np.ndarray): The point cloud as an Nx3 array of XYZ coordinates.
        voxel_size (float): The size of the voxels (uniform along all axes).
        
    Returns:
        voxel_indices (np.ndarray): The voxel indices for each point.
        voxel_grid (dict): A dictionary where keys are voxel indices (tuples) and values are points in that voxel.
    """
    # Normalize points by voxel size
    voxel_indices = np.floor(points[:,:3] / voxel_size).astype(np.int32)
    
    # Create a dictionary to store points in each voxel
    voxel_grid = {}
    
    for idx, voxel in tqdm(enumerate(voxel_indices)):
        voxel_key = tuple(voxel)  # Use tuple to make it hashable for the dictionary
        if voxel_key not in voxel_grid:
            voxel_grid[voxel_key] = []
        voxel_grid[voxel_key].append(points[idx])
    
    # Convert lists to arrays
    for key in voxel_grid:
        voxel_grid[key] = np.array(voxel_grid[key])
    
    return voxel_indices, voxel_grid

In [21]:
from tqdm import tqdm
#point_cloud = np.random.rand(1000, 3) * 100  # 1000 points in a 100x100x100 space
voxel_size = 20.0  # Voxel size of 5 units

# Voxelize the point cloud
voxel_indices, voxel_grid = voxelize_pointcloud(pts_all["Lille1_1.ply"], voxel_size)

30033430it [01:03, 469858.34it/s]


Voxel indices:
 [[1 4 2]
 [1 4 2]
 [1 4 2]
 [1 4 2]
 [1 4 2]
 [1 4 2]
 [1 4 2]
 [1 4 2]
 [1 4 2]
 [1 4 2]]

Number of unique voxels: 147


In [27]:
# Print some results
print("Voxel indices:\n", voxel_indices[900])  # Show first 10 voxel indices
print("\nNumber of unique voxels:", len(voxel_grid))

Voxel indices:
 [2 4 2]

Number of unique voxels: 147


In [35]:
for i in list(voxel_grid.keys()):
    print("\nPoints in a specific voxel:", len(voxel_grid[i]))
    print(np.bincount(voxel_grid[i][:,4].astype(np.int64)))


Points in a specific voxel: 137409
[   112   7322 124611      0      0   5252      0      0    112]

Points in a specific voxel: 538482
[  3266 340328 153348   4025      0      0   5988   1446  30081]

Points in a specific voxel: 181
[50 92 39]

Points in a specific voxel: 25
[ 0 25]

Points in a specific voxel: 247
[  4   0 243]

Points in a specific voxel: 6
[0 0 6]

Points in a specific voxel: 522670
[   510 222365 242290    382      0   6939      0      0  50184]

Points in a specific voxel: 846415
[  1142 370004 389402   1332      0   8646      0      0  75889]

Points in a specific voxel: 122
[  0   0 122]

Points in a specific voxel: 51089
[   61  1504 49479     0     0     0    45]

Points in a specific voxel: 960760
[   440 586447 255016   1551      0   2557   4577      0 104709   5463]

Points in a specific voxel: 287051
[  6363  55528 157370   3852      0      0  17028   1862   8393  36655]

Points in a specific voxel: 697
[  0   0  45   0   0   0   0   0   0 652]

Points i

In [30]:
print(voxel_grid[])

TypeError: 'dict_keys' object is not subscriptable